In [ ]:
import pandas as pd
import numpy as np, random
import itertools
from collections import Counter
from sklearn.model_selection import train_test_split
from gensim.models import KeyedVectors
import datetime
from keras.models import Sequential
from keras.layers import Input, Embedding, Activation, Dense, Concatenate, Reshape
from keras.layers.recurrent import LSTM
from keras.layers import Bidirectional
import keras.backend as K
from keras.callbacks import Callback,ModelCheckpoint, EarlyStopping
from time import time
from keras.models import load_model, Model

In [ ]:
from google.colab import drive
drive.mount('/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive/


In [ ]:
file = '/gdrive/My Drive/NLA_project/test_data.csv' 
data = pd.read_csv(file)

In [ ]:
data = data[['Cit_Text', 'Ref_text', 'label']]
print(len(data[data['label']==1]),len(data[data['label']==0]))
data.head()

15492 0


,Cit_Text,Ref_text,label
0,levin s classification extended nlp researcher...,augmented existing database levin semantic cla...,1
1,levin s classification extended nlp researcher...,augmented existing database levin semantic cla...,1
2,dang et al,augmented existing database levin semantic cla...,1
3,modify adding new class remove overlap class o...,augmented existing database levin semantic cla...,1
4,levin s classification extended nlp researcher...,augmented existing database levin semantic cla...,1


In [ ]:
#### CONSIDER REMOVING SENTENCE LENGTHS ABOVE 100
lower_lim = 3
upper_lim = 40


a, b, c = (data.values)[0]
print(str(a), str(b), str(c))

lis = list()
for a,b,c in data.values:
    a = str(a).split(' ')
    b = str(b).split(' ')
    
    if len(a) < lower_lim or len(b) < lower_lim:
        continue
    if len(a) > upper_lim:
        a = a[:upper_lim]
    if len(b) > upper_lim:
        b = b[:upper_lim]
        
    lis.append((' '.join(a),' '.join(b),c))

print(len(lis),len(data))
data = pd.DataFrame(lis , columns = ['Cit_Text', 'Ref_text', 'label'])
print(len(data[data['label']==1]),len(data[data['label']==0]))
data.head()

levin s classification extended nlp researcher dorr jones dang et al augmented existing database levin semantic class set intersective class created grouping to gether subset existing class over lapping member 1
15386 15492
15386 0


,Cit_Text,Ref_text,label
0,levin s classification extended nlp researcher...,augmented existing database levin semantic cla...,1
1,levin s classification extended nlp researcher...,augmented existing database levin semantic cla...,1
2,dang et al,augmented existing database levin semantic cla...,1
3,modify adding new class remove overlap class o...,augmented existing database levin semantic cla...,1
4,levin s classification extended nlp researcher...,augmented existing database levin semantic cla...,1


In [ ]:
### CROSS VALIDATION

# X1 = data['Cit_Text'].values
# X2 = data['Ref_text'].values
# y=data['label'].values

# skf = StratifiedKFold(n_splits=2)
# skf.get_n_splits(X, y)

# print(skf)

# for train_index, test_index in skf.split(X, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]

In [ ]:
max_len = 41
def load_data_and_labels():

    cit_text = data['Cit_Text'].values
    ref_text = data['Ref_text'].values

    cit_text = [str(temp).split(' ') for temp in cit_text]
    ref_text = [str(temp).split(' ') for temp in ref_text]
    
    x_text = [zipped for zipped in zip(cit_text, ref_text)]
    return x_text, data['label'].values

def pad_sentences(sentences, padding_word="<PAD/>"):
    
    global max_len
    max_len = max(max(len(x[0]), len(x[1])) for x in sentences)
    min_len =  min(min(len(x[0]), len(x[1])) for x in sentences)
    max_len += 1

    padded_sentences = list()
    for a,b in sentences:
        a += [padding_word] * (max_len - len(a))
        b += [padding_word] * (max_len - len(b))
        padded_sentences.append((a, b))
    return padded_sentences

def build_vocab(sentences):
    
    cit = list()
    ref = list()
    for a, b in sentences:
        cit.append(a)
        ref.append(b)
    word_counts = Counter(itertools.chain(*cit))
    dict.update(word_counts, Counter(itertools.chain(*ref)))

    vocabulary_inv = [x[0] for x in word_counts.most_common()]
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return [vocabulary, vocabulary_inv]

def build_input_data(sentences, labels, vocabulary):
    x1 = []
    for sentence in sentences:
        temp = []
        for word in sentence[0]:
            if str(word) in vocabulary:
                temp.append(vocabulary[str(word)])
            else:
                temp.append(random.randint(0, max_len))
        x1.append(temp)
    x2 = []
    for sentence in sentences:
        temp = []
        for word in sentence[1]:
            if str(word) in vocabulary:
                temp.append(vocabulary[str(word)])
            else:
                temp.append(random.randint(0, max_len))
        x2.append(temp)
    # x1 = np.array([[vocabulary[str(word)] for word in sentence[0]] for sentence in sentences])
    # x2 = np.array([[vocabulary[str(word)] for word in sentence[1]] for sentence in sentences])
    y = np.array(labels)
    return [np.array(x1), np.array(x2), y]


def load_data():

    sentences, labels = load_data_and_labels()
    
    sentences = pad_sentences(sentences)
    print(len(sentences), len(sentences[0][0]))
    # vocabulary, vocabulary_inv = build_vocab(sentences)
    X1, X2, y = build_input_data(sentences, labels, vocabulary)
    print(X1[:2], X2[:2], labels[:2])
    # return [X1, X2, y, vocabulary, vocabulary_inv]
    return X1,X2,y

X1, X2, y = load_data()
# X1, X2, y, vocabulary, vocabulary_inv = load_data()

15386 41
[[ 227  420  487 1623 1673 3170 3443 3297  222   30   29    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0]
 [ 227  420  487 1623 1673 3170 3443 3297  222   30   29    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0]] [[ 302  557  754  227   25   37   13  669   37  590 1003   37 1218  472
   557   37  872 1219 1081    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0]
 [ 302  557  754  227   25   37   13  669   37  590 1003   26 1218  472
   557   37  872 1219 1081    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0]] [1 1]


In [ ]:
# EMBEDDING_FILE = '..../GoogleNews-vectors-negative300.bin.gz'
# word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)
embeddings[0] = 0  # So that the padding will be ignored

for word, index in vocabulary.items():
#     if word in word2vec.vocab:
#         embeddings[index] = word2vec.word_vec(word)
#     else:
    embeddings[index] = np.random.uniform(-0.25,0.25,embedding_dim)
# del word2vec
print(embeddings.shape)

(5974, 300)


In [ ]:
X = list()
for a, b in zip(X1, X2):
    X.append((a, b))
X_train,X_test, Y_train, Y_test =  train_test_split(X, y,test_size = 0.20,random_state= 4)

X_train1 = list()
X_train2 = list()

X_test1 = list()
X_test2 = list()

for a, b in X_train:
    X_train1.append(a)
    X_train2.append(b)
    
for a, b in X_test:
    X_test1.append(a)
    X_test2.append(b)
    
X_train1 = np.array(X_train1)
X_train2 = np.array(X_train2)
X_test1 = np.array(X_test1)
X_test2 = np.array(X_test2)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)
print(len(X_train1), len(X_train2), len(Y_train))

46731 46731 46731


In [ ]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    # precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    # f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return recall
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    # recall = true_positives / (possible_positives + K.epsilon())
    # f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return precision
  

In [ ]:
# n_epoch = 5

# left_input = Input(shape=(max_len,))
# right_input = Input(shape=(max_len,))

# embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_len, trainable=True)

# encoded_left = embedding_layer(left_input)
# encoded_right = embedding_layer(right_input)

# # first_model = Sequential()
# # first_model.add(Bidirectional (LSTM (150, return_sequences=True, dropout=0.50),merge_mode='concat'))
# # # first_model.add(LSTM(output_dim, input_shape=(m, input_dim)))

# # second_model = Sequential()
# # # second_model.add(LSTM(output_dim, input_shape=(n-m, input_dim)))
# # second_model.add(Bidirectional (LSTM (150, return_sequences=True, dropout=0.50),merge_mode='concat'))


# # model = Sequential()
# # model.add(Concatenate([first_model, second_model]))
# # model.add(Dense(100))
# # model.add(Dense(32))
# # model.add(Dense(1))
# # model.add(Activation('sigmoid'))
# # model.compile('adam', 'binary_crossentropy', metrics=['accuracy', get_f1])

# mode_path = '/gdrive/My Drive/NLA_project/models/two_lstms.h5'
# training_start_time = time()

# # model.fit([X_train1, X_train2], Y_train, epochs=n_epoch, validation_split=0.2, shuffle=True)

# first_out = Bidirectional (LSTM (150, return_sequences=False, dropout=0.50),merge_mode='concat')(encoded_left)
# # first_out = Dense(1)(first_out)
# second_out = Bidirectional (LSTM (150, return_sequences=False, dropout=0.50),merge_mode='concat')(encoded_right)
# # second_out = Dense(1)(second_out)

# merged = Concatenate(axis=1)([first_out, second_out])
# # merged = Reshape([1,-1])(merged)
# output_layer1 = Dense(128)(merged)
# output_layer2 = Dense(32)(output_layer1)
# output_layer3 = Dense(1, activation='sigmoid')(output_layer2)
# # output_layer1 = Reshape([None, 1])(output_layer1)
# model = Model(inputs=[left_input, right_input], outputs=output_layer3)
# model.compile('adam', 'binary_crossentropy', metrics=['accuracy', get_f1, recall, precision])
# model.summary()

# early_stopping = EarlyStopping(monitor="val_loss", patience=5)
# # model.fit([X_train1, X_train2], Y_train, epochs=n_epoch, validation_split=0.3, shuffle=True,callbacks=[early_stopping, ModelCheckpoint(filepath=mode_path, save_best_only=True)])
# print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

saved_model = load_model(mode_path, custom_objects={"get_f1":get_f1, "precision":precision, "recall":recall})
model.compile('adam', 'binary_crossentropy', metrics=['accuracy', get_f1, recall, precision])
result = model.predict([X1, X2])
ans=0
for res in result:
  if res>0.5:
    ans+=1
print(ans, result)
model.evaluate([X1, X2], y)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


14168 [[9.999887e-01]
 [9.999888e-01]
 [9.982508e-01]
 ...
 [9.007040e-05]
 [9.786330e-01]
 [2.692158e-02]]
15386/15386 [==============================] - 19s 1ms/step


[0.5876500380610332,
 0.9208371043205261,
 0.9575027823448181,
 0.9208080172538757,
 1.0]